In [1]:
import torch

In [2]:
print("Importing Model")
import intent_model
print("Model Imported")
print("Importing Modules")
from weather import get_weather
from joke import get_joke
from dictionary import get_meaning
from horoscope import get_horoscope
from translation import translate_text
from sudoku import open_sudoku_window
print("Modules Imported")

Importing Model
Model Imported
Importing Modules
Modules Imported


In [3]:
import json

# Load and parse the JSON file
with open('intents_en.json', 'r') as file:
    intents_json = json.load(file)

In [4]:
import random

def get_response(tag):
    for intent in intents_json['intents']:
        if tag == intent['tag']:
            return random.choice(intent['responses'])

In [5]:
# Dictionary mapping language names to their short forms
languages = {
    "afrikaans": "af", "amharic": "am", "arabic": "ar", "asturian": "ast", 
    "azerbaijani": "az", "bashkir": "ba", "belarusian": "be", "bulgarian": "bg", 
    "bengali": "bn", "breton": "br", "bosnian": "bs", "catalan; valencian": "ca", 
    "cebuano": "ceb", "czech": "cs", "welsh": "cy", "danish": "da", 
    "german": "de", "greeek": "el", "english": "en", "spanish": "es", 
    "estonian": "et", "persian": "fa", "fulah": "ff", "finnish": "fi", 
    "french": "fr", "western frisian": "fy", "irish": "ga", "gaelic; scottish gaelic": "gd", 
    "galician": "gl", "gujarati": "gu", "hausa": "ha", "hebrew": "he", 
    "hindi": "hi", "croatian": "hr", "haitian; haitian creole": "ht", "hungarian": "hu", 
    "armenian": "hy", "indonesian": "id", "igbo": "ig", "iloko": "ilo", 
    "icelandic": "is", "italian": "it", "japanese": "ja", "javanese": "jv", 
    "georgian": "ka", "kazakh": "kk", "central khmer": "km", "kannada": "kn", 
    "korean": "ko", "luxembourgish; letzeburgesch": "lb", "ganda": "lg", "lingala": "ln", 
    "lao": "lo", "lithuanian": "lt", "latvian": "lv", "malagasy": "mg", 
    "macedonian": "mk", "malayalam": "ml", "mongolian": "mn", "marathi": "mr", 
    "malay": "ms", "burmese": "my", "nepali": "ne", "dutch; flemish": "nl", 
    "norwegian": "no", "northern sotho": "ns", "occitan (post 1500)": "oc", "oriya": "or", 
    "panjabi; punjabi": "pa", "polish": "pl", "pushto; pashto": "ps", "portuguese": "pt", 
    "romanian; moldavian; moldovan": "ro", "russian": "ru", "sindhi": "sd", 
    "sinhala; sinhalese": "si", "slovak": "sk", "slovenian": "sl", "somali": "so", 
    "albanian": "sq", "serbian": "sr", "swati": "ss", "sundanese": "su", 
    "swedish": "sv", "swahili": "sw", "tamil": "ta", "thai": "th", 
    "tagalog": "tl", "tswana": "tn", "turkish": "tr", "ukrainian": "uk", 
    "urdu": "ur", "uzbek": "uz", "vietnamese": "vi", "wolof": "wo", 
    "xhosa": "xh", "yiddish": "yi", "yoruba": "yo", "chinese": "zh", "zulu": "zu"
}

In [6]:
def get_key_from_value(d, value):
    for key, val in d.items():
        if val == value:
            return key
    return None 

In [7]:
def language_selector(language):
    # Check if the language is in the dictionary
    if language in languages.values():
        return language
    elif language in languages:
        return languages[language]
    else:
        return None

In [23]:
import tkinter as tk
from tkinter import scrolledtext

def initialize_chat():
    conversation_area.config(state=tk.NORMAL)  # Temporarily make it normal to insert text
    conversation_area.insert(tk.END,"""    
   _____              _    _   _      
  / ____|     /\     | |  | | | |     
 | (___      /  \    | |  | | | |     
  \___ \    / /\ \   | |  | | | |     
  ____) |  / ____ \  | |__| | | |____ 
 |_____/  /_/    \_\  \____/  |______|
           THE CHAT BOT
    """)
    global expecting
    expecting = "source_language"
    # conversation_area.insert(tk.END, "\nSaul: Hi, I am Saul!\nEnter Input Language or Code")
    conversation_area.insert(tk.END, "\nEnter Input Language or Code")
    scroll_to_end()
    conversation_area.config(state=tk.DISABLED)  # Set back to disabled to make it read-only

def process_language_selection(user_input):
    global src_lang, tgt_lang, expecting

    if expecting == "source_language":
        if language_selector(user_input.strip().lower()):
            src_lang = language_selector(user_input.strip().lower())
            conversation_area.insert(tk.END, "\nEnter Output Language or Code")
            scroll_to_end()
            expecting = "target_language"
        else:
            conversation_area.insert(tk.END, "\nSaul: Language Not Found, Enter Again!")
            scroll_to_end()
    elif expecting == "target_language":
        if language_selector(user_input.strip().lower()):
            tgt_lang = language_selector(user_input.strip().lower())
            src_full = get_key_from_value(languages, src_lang)
            tgt_full = get_key_from_value(languages, tgt_lang)
            conversation_area.insert(tk.END, \
                                     f"\nSaul: Set Input in {src_full.capitalize()} and Output in {tgt_full.capitalize()}")
            conversation_area.insert(tk.END, \
                                     f"\nSaul: {translate_text('Hi! How may I help you?', src_lang, tgt_lang)}")
            scroll_to_end()
            expecting = None
        else:
            conversation_area.insert(tk.END, "\nSaul: Language Not Found, Enter Again!")

def send_message(event=None):
    global expecting
    conversation_area.config(state=tk.NORMAL)  # Temporarily make it normal to insert text
    user_text = user_input.get()
    user_input.delete(0, tk.END)
    response = str()
    try:
        if expecting in ["source_language", "target_language"]:
            conversation_area.insert(tk.END, f"\nUser: {user_text} ")
            process_language_selection(user_text)
        elif expecting in ["weather", "horoscope", "dictionary", "pass"]:
            if expecting == "weather":
                response = f"The temperature is {get_weather(user_text)}F"
                expecting = None
            elif expecting == "horoscope":
                response = get_horoscope(user_text)
                expecting = None
            elif expecting == "dictionary":
                response = get_meaning(user_text)
                expecting = None
            else:
                pass
            
            if tgt_lang != "en":
                conversation_area.insert(tk.END, "\nUser: " + translate_text(user_text, 'en', tgt_lang))
                conversation_area.insert(tk.END, "\nSaul: " + translate_text(response, 'en', tgt_lang))
                conversation_area.insert(tk.END, "\nSaul: " + translate_text(get_response('pass'), 'en', tgt_lang))
            else:
                conversation_area.insert(tk.END, "\nUser: " + user_text)
                conversation_area.insert(tk.END, "\nSaul: " + response)
                conversation_area.insert(tk.END, "\nSaul: " + get_response('pass'))
            scroll_to_end()
        else:
            conversation_area.insert(tk.END, "\nUser: " + user_text)
            sentence = translate_text(user_text, src_lang, "en")
            tag = intent_model.predict(intent_model.model\
                                       , sentence\
                                       , intent_model.all_words\
                                       , intent_model.tags\
                                       , intent_model.device)
            print(f"DEBUG: Predicted tag: {tag}")
            if tag == "joke":
                response = get_joke()
            elif tag == "sudoku":
                if tgt_lang != "en":
                    conversation_area.insert(tk.END, "\nSaul: " + translate_text("Opened in a new window", 'en', tgt_lang))
                    conversation_area.insert(tk.END, "\nSaul: " + translate_text(get_response('pass'), 'en', tgt_lang))
                else:
                    conversation_area.insert(tk.END, "\nSaul: " + "Opened in a new window")
                    conversation_area.insert(tk.END, "\nSaul: " + get_response('pass'))
                open_sudoku_window()
            elif tag == "weather":
                response = 'Enter City, State'
                expecting = "weather"
            elif tag == "horoscope":
                response = 'Enter your Sunshine'
                expecting = "horoscope"
            elif tag == "dictionary":
                response = "Enter The Word"
                expecting = "dictionary"
            elif tag == "goodbye":
                response = get_response(tag)
                response += "\n\n      In Legal Problem?\n      Better Call Saul!"
                disable_chat()
            else:
                response = get_response(tag)
                
            if tgt_lang != "en":
                conversation_area.insert(tk.END, "\nSaul: " + translate_text(response, 'en', tgt_lang))
            else:
                conversation_area.insert(tk.END, "\nSaul: " + response)
                
            if tag == "joke" or tag == "sudoku":
                if tgt_lang != "en":
                    conversation_area.insert(tk.END, "\nSaul: " + translate_text(get_response('pass'), 'en', tgt_lang))
                else:
                    conversation_area.insert(tk.END, "\nSaul: " + get_response('pass'))
            scroll_to_end()
    except:
        if tgt_lang != "en":
            conversation_area.insert(tk.END, "\nSaul: " + translate_text('Something went wrong!', 'en', tgt_lang))
            conversation_area.insert(tk.END, "\nSaul: " + translate_text(get_response('pass'), 'en', tgt_lang))
        else:
            conversation_area.insert(tk.END, "\nSaul: Something went wrong!")
            conversation_area.insert(tk.END, "\nSaul: " + get_response('pass'))
        scroll_to_end()
        expecting = None
    finally:
        conversation_area.config(state=tk.DISABLED)  # Set back to disabled to make it read-only

def scroll_to_end():
    conversation_area.see(tk.END)

def disable_chat():
    user_input.config(state=tk.DISABLED, bg="light grey")
    send_button.config(state=tk.DISABLED, bg="dark grey")
    
# Main window
window = tk.Tk()
window.title("Saul, The Chat Bot")

# Conversation display area
conversation_area = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=60, height=40)
conversation_area.grid(column=0, row=0, columnspan=4, padx=10, pady=10)
conversation_area.config(state=tk.DISABLED)  # Make the text area read-only

# Label for user input
user_label = tk.Label(window, text="User:")
user_label.grid(column=0, row=1, sticky='e', padx=(10, 0), pady=10)

# User input text field
user_input = tk.Entry(window, width=30)
user_input.grid(column=1, row=1, padx=(0, 10), pady=10)
user_input.bind("<Return>", send_message)

# Send button
send_button = tk.Button(window, text="Send", command=send_message)
send_button.grid(column=2, row=1)

# Exit button
close_app_button = tk.Button(window, text="Exit", command=window.destroy)
close_app_button.grid(column=3, row=1)

# Global variables for source and target languages and state
src_lang = None
tgt_lang = None
expecting = None

# Initialize the chat
initialize_chat()

# Run the application
window.mainloop()

DEBUG: Predicted tag: goodbye
